In [1]:
import new_adventure as na
import new_adventure.derivative_free_estimation as dfe
import numpy as np
import matplotlib.pyplot as plt
import time

import numpy as np
import time

import multiprocessing
from os import getpid

%load_ext autoreload
%autoreload 2


/Users/daniellengyel/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [6]:
def f(i):
    print(i)
    return 1

def process_task(x_0, F, process_N, alpha, L_sample_points, L_grads, pid):
    x_0 = np.ones(dim) / np.linalg.norm(np.ones(dim))

#     print(process_N)
#     print(pid)
    # Notice, we should be sharing the radius. So some lock is probably needed to synchronise 
    sample_points = dfe.hit_run(x_0, F, x_0.shape[0], process_N, alpha)
    out_grads = F.f1(sample_points)
#     L_sample_points.append(sample_points)
#     L_grads.append(out_grads)

def new_beta_second_shift_estimator(F, x_0, alpha, N, control_variate=True, num_processes=1):



#     with multiprocessing.Manager() as manager:
    manager = multiprocessing.Manager()
    pool = multiprocessing.Pool(processes=num_processes)
    L_sample_points = None # [] #manager.list()
    L_grads = None # [] #manager.list()
    pool_workers = []
    for i in range(num_processes):
#             p = pool.apply_async(f, (i,))
        p = pool.apply_async(process_task, (None, F, N // num_processes, alpha, L_sample_points, L_grads, i,))
        pool_workers.append(p)
#         p.wait()
#             print(i)
#         print(pool_workers)
    for pw in pool_workers:
        a = time.time()
        pw.wait()
#         print(time.time() - a)
#     [pw.wait() for pw in pool_workers]
#         print([pw.get(timeout=1) for pw in pool_workers])


#         sample_points = np.vstack(list(L_sample_points))
#         out_grads = np.vstack(list(L_grads))

#     second_shift_est = dfe.new_proper_cov(sample_points, out_grads)
#     return second_shift_est.dot(np.linalg.inv(dfe.np_new_cov(sample_points)))

In [7]:
np.random.seed(10)
dim = 2500
num_barriers = dim * 2
dirs = np.random.normal(size=(num_barriers, dim)) # sample gaussian and normalize 
ws = dirs/np.linalg.norm(dirs, axis=1).reshape(-1, 1)
bs = np.ones(num_barriers)

barrier = na.Barriers.LogPolytopeBarrier(ws, bs)

F = na.Functions.Linear(np.ones(dim))

F = barrier # na.Functions.LinearCombination(F, barrier, [1, 1])

In [9]:
xs = np.ones(dim) / np.linalg.norm(np.ones(dim))

a = time.time()
new_beta_second_shift_estimator(F, xs, 200, 10000, control_variate=True, num_processes=1)
print(time.time() - a)

a = time.time()
new_beta_second_shift_estimator(F, xs, 200, 10000, control_variate=True, num_processes=2)
print(time.time() - a)

11.348680019378662
9.809731245040894


Process ForkPoolWorker-32:
Process ForkPoolWorker-34:
Process ForkPoolWorker-35:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/daniellengyel/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/daniellengyel/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/daniellengyel/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/daniellengyel/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/daniellengyel/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/daniellengyel/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/User

In [56]:
a = time.time()
dfe.new_beta_second_shift_estimator(F, xs, 200, 10000, control_variate=True)
print(time.time() - a)

9.42070984840393


In [147]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    print(x)
    return x*x


pool = multiprocessing.Pool(4)
    

# launching multiple evaluations asynchronously *may* use more processes
multiple_results = [pool.apply_async(f, (i,)) for i in range(4)]
# print([res.get(timeout=1) for res in multiple_results])

# res = pool.apply_async(f, (20,))      # runs in *only* one process
# print(res.get(timeout=1))             # prints "400"

1
2
0
3


Process ForkPoolWorker-286:
Process ForkPoolWorker-288:
Traceback (most recent call last):


In [12]:
from multiprocessing import Process, Queue
a = time.time()
def sum(a,b):
    su=0
    for i in range(a,b):
        su+=i
    q.put(su)

q= Queue()

p1=Process(target=sum, args=(1,25*10**7))
p2=Process(target=sum, args=(25*10**7,5*10**8))
p3=Process(target=sum, args=(5*10**8,75*10**7))
p4=Process(target=sum, args=(75*10**7,10**9))

p1.start()
p2.start()
p3.start()
p4.start()

r1=q.get()
r2=q.get()
r3=q.get()
r4=q.get()

print(r1+r2+r3+r4)
print(time.time() - a)

499999999500000000
13.995182991027832


In [ ]:
def sum(a,b):
    su=0
    for i in range(a,b):
        su+=i
    print(su)
a = time.time()
sum(1,10**9)
print(time.time() - a)